# 1. Imports

In [1]:
import copy
import numba
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
import pandas as pd
import seaborn as sns 
import statistics
%matplotlib inline 
from os import getcwd
import plotly.express as px
import matplotlib.cm as cm
import pylab as pl

from scipy import sparse
from scipy import stats
from scipy import spatial

import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics.pairwise import cosine_similarity

from surprise import accuracy
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic

getcwd()

'C:\\Users\\tessa\\UVT Thesis\\Untitled Folder'

# 2. Preprocessing

In [2]:
data_inner = pd.read_csv('data_inner_preprocessed.csv')
testdata = pd.read_csv('testdata_inner.csv')

In [3]:
# 12 variables
data_inner = data_inner[['loudness', 'speechiness', 'acousticness', 'duration_ms', 'instrumentalness', 'danceability',
                         'energy', 'chorus_hit','mode','valence','sections','key',
                         'song_id', 'user_id', 
                         'rel_rating']]
testdata = testdata[['loudness', 'speechiness', 'acousticness', 'duration_ms', 'instrumentalness', 'danceability',
                         'energy','chorus_hit','mode','valence','sections','key',
                         'song_id', 'user_id', 
                         'rel_rating']]

dataset_for_clustering = data_inner[['loudness', 'speechiness', 'acousticness', 'duration_ms', 'instrumentalness', 'danceability',
                         'energy', 'chorus_hit', 'valence','sections','key','mode']]
testdata_for_clustering = testdata[['loudness', 'speechiness', 'acousticness', 'duration_ms', 'instrumentalness', 'danceability',
                         'energy', 'chorus_hit', 'valence','sections','key','mode']]

# 3. Clustering

In [4]:
# make the clusters
km = KMeans(n_clusters=6, init='k-means++', random_state=42)
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', km)],verbose=True)

song_cluster_pipeline.fit(dataset_for_clustering)
song_cluster_labels = song_cluster_pipeline.predict(dataset_for_clustering)
data_inner['cluster'] = song_cluster_labels

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.4s
[Pipeline] ............ (step 2 of 2) Processing kmeans, total=  22.9s


In [5]:
cluster0 = data_inner.loc[data_inner['cluster'] == 0]
print(cluster0.shape)

cluster1 = data_inner.loc[data_inner['cluster'] == 1]
print(cluster1.shape)

cluster2 = data_inner.loc[data_inner['cluster'] == 2]
print(cluster2.shape)

cluster3 = data_inner.loc[data_inner['cluster'] == 3]
print(cluster3.shape)

cluster4 = data_inner.loc[data_inner['cluster'] == 4]
print(cluster4.shape)

cluster5 = data_inner.loc[data_inner['cluster'] == 5]
print(cluster5.shape)

(102716, 16)
(134395, 16)
(47875, 16)
(88429, 16)
(33948, 16)
(243663, 16)


In [6]:
test_song_cluster_labels = song_cluster_pipeline.predict(testdata_for_clustering)

# assign cluster to testdata
testdata['cluster'] = test_song_cluster_labels

In [7]:
# split test clusters
# test clusters
testcluster0 = testdata.loc[testdata['cluster'] == 0]
print(testcluster0.shape)

testcluster1 = testdata.loc[testdata['cluster'] == 1]
print(testcluster1.shape)

testcluster2 = testdata.loc[testdata['cluster'] == 2]
print(testcluster2.shape)

testcluster3 = testdata.loc[testdata['cluster'] == 3]
print(testcluster3.shape)

testcluster4 = testdata.loc[testdata['cluster'] == 4]
print(testcluster4.shape)

testcluster5 = testdata.loc[testdata['cluster'] == 5]
print(testcluster5.shape)

(26178, 16)
(33590, 16)
(12044, 16)
(22064, 16)
(8237, 16)
(61139, 16)


# Cluster analysis

In [8]:
# Calinski Harabasz score
from sklearn import metrics
from sklearn.metrics import pairwise_distances

labels_train = data_inner['cluster']
X_train = dataset_for_clustering
labels_test = testdata['cluster']
X_test = testdata_for_clustering

print(metrics.calinski_harabasz_score(X_train, labels_train))
print(metrics.calinski_harabasz_score(X_test, labels_test))

# train: 147470.27258197733
# test: 37189.521980430334

94985.93808920744
23729.91381617131


In [9]:
# Davies Bouldin score
from sklearn.metrics import davies_bouldin_score

print(davies_bouldin_score(X_train, labels_train))
print(davies_bouldin_score(X_test, labels_test))

# train: 1.4596846093446396
# test: 1.453712603628295

2.2165760649359894
2.216652034945887


In [10]:
data_inner.groupby("cluster").mean()

,loudness,speechiness,acousticness,duration_ms,instrumentalness,danceability,energy,chorus_hit,mode,valence,sections,key,rel_rating
cluster,,,,,,,,,,,,,
0,0.780990,0.035474,0.127533,0.459676,0.018798,0.467359,0.686956,0.213245,0.799330,0.375940,0.406801,0.441371,0.967372
1,0.822101,0.057170,0.114130,0.329898,0.011015,0.657440,0.745077,0.224086,0.000000,0.623231,0.276530,0.558479,0.952444
2,0.837422,0.397839,0.172094,0.352996,0.002196,0.665386,0.728918,0.227207,0.699572,0.588804,0.291642,0.367835,0.931147
3,0.654612,0.025475,0.608761,0.314057,0.010161,0.507251,0.402128,0.229224,0.882866,0.439747,0.273357,0.475681,0.984110
4,0.650857,0.051867,0.296646,0.416441,0.789767,0.440231,0.618512,0.237902,0.432161,0.356111,0.333891,0.520178,1.013867
5,0.832387,0.044922,0.091456,0.313989,0.013183,0.597824,0.764281,0.236311,1.000000,0.597766,0.261131,0.459702,0.982129


# 4. Weights

In [11]:
weight_c0 = cluster0.shape[0] / data_inner.shape[0]
weight_c1 = cluster1.shape[0] / data_inner.shape[0]
weight_c2 = cluster2.shape[0] / data_inner.shape[0]
weight_c3 = cluster3.shape[0] / data_inner.shape[0]
weight_c4 = cluster4.shape[0] / data_inner.shape[0]
weight_c5 = cluster5.shape[0] / data_inner.shape[0]

weight_tc0 = testcluster0.shape[0] / testdata.shape[0]
weight_tc1 = testcluster1.shape[0] / testdata.shape[0]
weight_tc2 = testcluster2.shape[0] / testdata.shape[0]
weight_tc3 = testcluster3.shape[0] / testdata.shape[0]
weight_tc4 = testcluster4.shape[0] / testdata.shape[0]
weight_tc5 = testcluster5.shape[0] / testdata.shape[0]

# 5. KNN

In [12]:
sim_options = {'name': 'cosine',
              'user_based': False}

model_knn = KNNBasic(sim_options=sim_options, k=14, verbose=True)

In [13]:
reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
cluster0 = Dataset.load_from_df(cluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster1 = Dataset.load_from_df(cluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster2 = Dataset.load_from_df(cluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster3 = Dataset.load_from_df(cluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster4 = Dataset.load_from_df(cluster4[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster5 = Dataset.load_from_df(cluster5[['user_id', 'song_id', 'rel_rating']], reader = reader)

type(cluster0)

from surprise.model_selection import train_test_split

train_cluster0, val_cluster0 = train_test_split(cluster0, test_size=0.25, random_state=42)
train_cluster1, val_cluster1 = train_test_split(cluster1, test_size=0.25, random_state=42)
train_cluster2, val_cluster2 = train_test_split(cluster2, test_size=0.25, random_state=42)
train_cluster3, val_cluster3 = train_test_split(cluster3, test_size=0.25, random_state=42)
train_cluster4, val_cluster4 = train_test_split(cluster4, test_size=0.25, random_state=42)
train_cluster5, val_cluster5 = train_test_split(cluster5, test_size=0.25, random_state=42)
print(type(train_cluster0), type(val_cluster0))

# get train set in a different format
exc00, newtrain_cluster0 = train_test_split(cluster0, test_size=0.999, random_state=42)
exc11, newtrain_cluster1 = train_test_split(cluster1, test_size=0.999, random_state=42)
exc22, newtrain_cluster2 = train_test_split(cluster2, test_size=0.999, random_state=42)
exc33, newtrain_cluster3 = train_test_split(cluster3, test_size=0.999, random_state=42)
exc44, newtrain_cluster4 = train_test_split(cluster4, test_size=0.999, random_state=42)
exc55, newtrain_cluster5 = train_test_split(cluster5, test_size=0.999, random_state=42)

print(type(exc00), type(train_cluster0))

reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
test_cluster0 = Dataset.load_from_df(testcluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster1 = Dataset.load_from_df(testcluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster2 = Dataset.load_from_df(testcluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster3 = Dataset.load_from_df(testcluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster4 = Dataset.load_from_df(testcluster4[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster5 = Dataset.load_from_df(testcluster5[['user_id', 'song_id', 'rel_rating']], reader = reader)
type(test_cluster0)

# split data into test and excess dataset to get a different type
#from surprise.model_selection import train_test_split

exc0, test_cluster0 = train_test_split(test_cluster0, test_size=0.999, random_state=42)
exc1, test_cluster1 = train_test_split(test_cluster1, test_size=0.999, random_state=42)
exc2, test_cluster2 = train_test_split(test_cluster2, test_size=0.999, random_state=42)
exc3, test_cluster3 = train_test_split(test_cluster3, test_size=0.999, random_state=42)
exc4, test_cluster4 = train_test_split(test_cluster4, test_size=0.999, random_state=42)
exc5, test_cluster5 = train_test_split(test_cluster5, test_size=0.999, random_state=42)

<class 'surprise.trainset.Trainset'> <class 'list'>
<class 'surprise.trainset.Trainset'> <class 'surprise.trainset.Trainset'>


In [14]:
# find the train mae
prediction0_0 = model_knn.fit(train_cluster0).test(newtrain_cluster0)
prediction1_0 = model_knn.fit(train_cluster1).test(newtrain_cluster1)
prediction2_0 = model_knn.fit(train_cluster2).test(newtrain_cluster2)
prediction3_0 = model_knn.fit(train_cluster3).test(newtrain_cluster3)
prediction4_0 = model_knn.fit(train_cluster4).test(newtrain_cluster4)
prediction5_0 = model_knn.fit(train_cluster5).test(newtrain_cluster5)

# find the validation set mae
prediction0_1 = model_knn.fit(train_cluster0).test(val_cluster0)
prediction1_1 = model_knn.fit(train_cluster1).test(val_cluster1)
prediction2_1 = model_knn.fit(train_cluster2).test(val_cluster2)
prediction3_1 = model_knn.fit(train_cluster3).test(val_cluster3)
prediction4_1 = model_knn.fit(train_cluster4).test(val_cluster4)
prediction5_1 = model_knn.fit(train_cluster5).test(val_cluster5)

# check mae for test set for model comparison
prediction0_2 = model_knn.fit(train_cluster0).test(test_cluster0)
prediction1_2 = model_knn.fit(train_cluster1).test(test_cluster1)
prediction2_2 = model_knn.fit(train_cluster2).test(test_cluster2)
prediction3_2 = model_knn.fit(train_cluster3).test(test_cluster3)
prediction4_2 = model_knn.fit(train_cluster4).test(test_cluster4)
prediction5_2 = model_knn.fit(train_cluster5).test(test_cluster5)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

In [15]:
## average prediction frame

In [35]:
# TEST DATA
result0 = pd.DataFrame(prediction0_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

,visitor_id,item_id,base_event,predict_event,error
0,c6ef770da294053e586e983409fdfe255b4a1dcc,SOHTQAS12A6701C7BA,0.645161,0.645161,0.000000
1,fc54f6bb5a9bec25874b454c5969e58a91889f4d,SOLFXKT12AB017E3E0,3.891239,2.624366,1.266873
2,6145cb43498f39dd1a19734264237a4c49da08ba,SOZVCRW12A67ADA0B7,1.555556,1.351957,0.203598
3,5c9c438f176a7b72e7c44f901cd3423edb4ceefa,SONYKOW12AB01849C9,1.839572,0.857995,0.981577
4,b28fbda381ff1745846d7f1572c5b3b5a1b94d3f,SOJSRKL12A8159E8A8,0.054688,0.054688,0.000000


In [17]:
# TRAIN DATA
result0 = pd.DataFrame(prediction0_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

,visitor_id,item_id,base_event,predict_event,error
0,1f7108c4df0d32da781444a2c621e0b60de0f3b9,SOCGWCK12A67020F39,0.787879,1.229923,0.442044
1,9c859962257112ad523f1d3c121d35191daa6d2b,SOSJUAX12A58A7FD30,1.871940,1.353814,0.518126
2,2a1928f9b4633a643f870f08852c678df30d6d13,SOKWOBO12A8C1424F6,0.237288,1.267210,1.029922
3,969bb67e726e0256c743b6347eb37d8a00f949f5,SOTZION12A6D4FB477,1.500000,0.974829,0.525171
4,aaedd8f767748cb2fa175f87858c5c5c859bbfe1,SOIWPRZ12AB018244E,1.649123,1.096129,0.552993


In [18]:
# VALIDATION
result0 = pd.DataFrame(prediction0_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

,visitor_id,item_id,base_event,predict_event,error
0,6ce0cb08a60ecc3466da8d7cffadcc361a60e2d1,SOPOANU12A8AE48C9B,0.527778,2.638889,2.111111
1,e89fc89bd113485ad985724b048ceb4f53fd672a,SOIJVVR12A6701C2EE,0.488152,0.569463,0.081311
2,56818e964aa1cb5c9c3f13253f02a9da24fe95d1,SOSZJFV12AB01878CB,0.847826,1.052065,0.204239
3,71826d1b4dc839c9a93f81188c88879a05479d3c,SOFEOVT12A58A78DF3,1.750663,1.317828,0.432835
4,c756f89127324567c6a39835276a7c0400a481db,SOKOPJY12A8C13BEB7,0.318386,0.807551,0.489166


In [36]:
# do not merge

## result0
play_count_categories = []

for number in result0['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result0 = result0.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result0['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result0 = result0.assign(predict_binary = play_count_categories1)

result0.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,8b45417f337bd127366a12f1cc32ef94b6e3a5d3,SOUMWTG12A8C138407,1.254098,1.254098,0.000000,1,1
1,8a4fe8fe23cbf7bb269d7eb0f5528d7487fbfb33,SOUFPNI12A8C142D19,1.581818,1.631015,0.049197,1,1
2,12be039b66971ca076158bf1d48343ec0d7fb1cd,SOUFPNI12A8C142D19,0.547872,0.861086,0.313214,0,0
3,9fcc61eca268faa7fe79b91c7176f8322ce4cfa1,SOTSJDX12AF72A9AFD,0.863636,0.863636,0.000000,0,0
4,853abf16200538f5d3d2b011aff4d94e645adb3e,SOZUVYE12AB017CA2D,2.785388,1.008479,1.776910,1,1


In [37]:
# do not merge

## result1
play_count_categories = []

for number in result1['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result1 = result1.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result1['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result1 = result1.assign(predict_binary = play_count_categories1)

result1.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,0ef3fc780b2d3e3d95668a1d7d649b38c7c8e6c4,SOJWBYS12A67AE17A1,0.582090,2.066296,1.484207e+00,0,1
1,8b60c3b6c9174d4142d50797562642e731950847,SOLZYWQ12A8C13F269,1.190476,0.793651,3.968254e-01,1,0
2,f5aafb39976a4976002b01ad0c3fae6dedbe0df0,SOKOXWU12AF72AD1BC,1.173184,1.003440,1.697444e-01,1,1
3,2f11dcb59d301ed4848d2574f86f00b0106c2da5,SOVDLZN12AB0185BEA,0.892857,1.301894,4.090371e-01,0,1
4,2b660df1a28fbf38bf3f6445aa57cc9bb86c00f3,SOKUAGP12A8C133B94,0.454545,0.454545,5.551115e-17,0,0


In [38]:
# do not merge

## result2
play_count_categories = []

for number in result2['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result2 = result2.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result2['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result2 = result2.assign(predict_binary = play_count_categories1)

result2.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,22152a3af1c97b7a9af21f7ce199ba023fb79fe8,SODBMRI12A8151AF45,0.555556,0.490210,0.065345,0,0
1,87996fa5f01758b81cdff9d2fa6e680763e3d2e4,SOSJGZM12AF72A377E,0.666667,0.666667,0.000000,0,0
2,530384e5af3386538b4d8f8527ee5b4be31bd0a0,SOWCBKV12AC90732A6,0.203125,0.928803,0.725678,0,0
3,b8812e995a3120738728d91651c90447939d2e34,SODTHOY12AC9071AE7,0.184874,0.184874,0.000000,0,0
4,5ccabf570d942b5de1b9e46861c9109ee469d5dc,SOCHADN12A6310ED94,0.625000,0.625000,0.000000,0,0


In [39]:
# do not merge

## result3
play_count_categories = []

for number in result3['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result3 = result3.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result3['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result3 = result3.assign(predict_binary = play_count_categories1)

result3.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,ed1ca87f320789fb78ea321904df94e993887b54,SOJOYQG12A8C1443B1,0.572254,0.838450,0.266196,0,0
1,03e78c49f4799c02299fa15c42a506a57ba8e9a2,SOHTTNB12A8C1357E9,0.603774,0.603774,0.000000,0,0
2,cd495501d6fd6b73bd334c0c6809cadfc126994e,SOAWJSH12A8C13AE09,3.063830,0.982643,2.081187,1,0
3,4fc1762b86d03b7b2b90031246789fadafa40a8f,SOOICOR12A6D4FC106,0.312500,0.982643,0.670143,0,0
4,86450e6c2d8ebd7ead7a1a8ff098de0dd1808418,SORCAZW12A8C1353A4,0.744681,0.932736,0.188055,0,0


In [40]:
# do not merge

## result4
play_count_categories = []

for number in result4['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result4 = result4.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result4['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result4 = result4.assign(predict_binary = play_count_categories1)

result4.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,9c6f287c3004ab20b78e440f22d73038c7de753e,SOEPJEF12AF72A44DA,1.286408,1.286408,0.000000e+00,1,1
1,596480392c7998aa45ff0bf32a4968b8a5c8439d,SOMORYX12B0B80908A,0.751634,0.751634,1.110223e-16,0,0
2,83021d15f4ca72804530eb9cf8909d5228955570,SOFBBZQ12A8C134EDB,1.772973,1.772973,0.000000e+00,1,1
3,b0e24e03373c33cf987c709951c0c002376bb1e6,SOZVVRE12A8C143150,2.352941,1.288552,1.064389e+00,1,1
4,6b5eabe52df3552bec7199ef3068f60d6d9a33a5,SOGWIAD12AF72A7F7B,3.102041,3.102041,0.000000e+00,1,1


In [41]:
# do not merge

## result5
play_count_categories = []

for number in result5['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result5 = result5.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result5['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result5 = result5.assign(predict_binary = play_count_categories1)

result5.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,c6ef770da294053e586e983409fdfe255b4a1dcc,SOHTQAS12A6701C7BA,0.645161,0.645161,0.000000,0,0
1,fc54f6bb5a9bec25874b454c5969e58a91889f4d,SOLFXKT12AB017E3E0,3.891239,2.624366,1.266873,1,1
2,6145cb43498f39dd1a19734264237a4c49da08ba,SOZVCRW12A67ADA0B7,1.555556,1.351957,0.203598,1,1
3,5c9c438f176a7b72e7c44f901cd3423edb4ceefa,SONYKOW12AB01849C9,1.839572,0.857995,0.981577,1,0
4,b28fbda381ff1745846d7f1572c5b3b5a1b94d3f,SOJSRKL12A8159E8A8,0.054688,0.054688,0.000000,0,0


In [42]:
# calculate accuracy
from sklearn.metrics import confusion_matrix

y_actual0 = pd.Series(result0['base_binary'], name='actual')
y_predict0 = pd.Series(result0['predict_binary'], name='predicted')
con_mat0 = (pd.crosstab(y_actual0, y_predict0)) / (pd.crosstab(y_actual0, y_predict0)).sum(axis=1)

y_actual1 = pd.Series(result1['base_binary'], name='actual')
y_predict1 = pd.Series(result1['predict_binary'], name='predicted')
con_mat1 = (pd.crosstab(y_actual1, y_predict1)) / (pd.crosstab(y_actual1, y_predict1)).sum(axis=1)

y_actual2 = pd.Series(result2['base_binary'], name='actual')
y_predict2 = pd.Series(result2['predict_binary'], name='predicted')
con_mat2 = (pd.crosstab(y_actual2, y_predict2)) / (pd.crosstab(y_actual2, y_predict2)).sum(axis=1)

y_actual3 = pd.Series(result3['base_binary'], name='actual')
y_predict3 = pd.Series(result3['predict_binary'], name='predicted')
con_mat3 = (pd.crosstab(y_actual3, y_predict3)) / (pd.crosstab(y_actual3, y_predict3)).sum(axis=1)

y_actual4 = pd.Series(result4['base_binary'], name='actual')
y_predict4 = pd.Series(result4['predict_binary'], name='predicted')
con_mat4 = (pd.crosstab(y_actual4, y_predict4)) / (pd.crosstab(y_actual4, y_predict4)).sum(axis=1)

y_actual5 = pd.Series(result5['base_binary'], name='actual')
y_predict5 = pd.Series(result5['predict_binary'], name='predicted')
con_mat5 = (pd.crosstab(y_actual5, y_predict5)) / (pd.crosstab(y_actual5, y_predict5)).sum(axis=1)

In [43]:
print('Result 0:', con_mat0)
print('\n')

print('Result 1:', con_mat1)
print('\n')

print('Result 2:', con_mat2)
print('\n')

print('Result 3:', con_mat3)
print('\n')

print('Result 4:', con_mat4)
print('\n')

print('Result 5:', con_mat5)
print('\n')

Result 0: predicted         0         1
actual                       
0          0.843104  0.361374
1          0.133534  0.692434


Result 1: predicted         0         1
actual                       
0          0.829840  0.412677
1          0.138889  0.663162


Result 2: predicted         0         1
actual                       
0          0.889240  0.270983
1          0.131366  0.678602


Result 3: predicted         0         1
actual                       
0          0.839107  0.369689
1          0.145331  0.666068


Result 4: predicted         0         1
actual                       
0          0.761871  0.526131
1          0.063901  0.858814


Result 5: predicted         0         1
actual                       
0          0.768383  0.532038
1          0.153409  0.647611




In [44]:
TP = (con_mat0[1][1])*weight_tc0 + (con_mat1[1][1])*weight_tc1 + (con_mat2[1][1])*weight_tc2 + (con_mat3[1][1])*weight_tc3 + (con_mat4[1][1])*weight_tc4 + (con_mat5[1][1])*weight_tc5 
TN = (con_mat0[0][0])*weight_tc0 + (con_mat1[0][0])*weight_tc1 + (con_mat2[0][0])*weight_tc2 + (con_mat3[0][0])*weight_tc3 + (con_mat4[0][0])*weight_tc4 + (con_mat5[0][0])*weight_tc5 
FP = (con_mat0[0][1])*weight_tc0 + (con_mat1[0][1])*weight_tc1 + (con_mat2[0][1])*weight_tc2 + (con_mat3[0][1])*weight_tc3 + (con_mat4[0][1])*weight_tc4 + (con_mat5[0][1])*weight_tc5 
FN = (con_mat0[1][0])*weight_tc0 + (con_mat1[1][0])*weight_tc1 + (con_mat2[1][0])*weight_tc2 + (con_mat3[1][0])*weight_tc3 + (con_mat4[1][0])*weight_tc4 + (con_mat5[1][0])*weight_tc5 

print('TP:',TP,'\nTN:',TN,'\nFP:',FP,'\nFN:',FN)

TP: 0.6734357426612957 
TN: 0.8111561141564638 
FP: 0.14000023357653704 
FN: 0.4386127864639575


In [45]:
from sklearn import metrics
recall0 = metrics.recall_score(y_actual0, y_predict0)
recall1 = metrics.recall_score(y_actual1, y_predict1)
recall2 = metrics.recall_score(y_actual2, y_predict2)
recall3 = metrics.recall_score(y_actual3, y_predict3)
recall4 = metrics.recall_score(y_actual4, y_predict4)
recall5 = metrics.recall_score(y_actual5, y_predict5)
recall = recall0*weight_tc0 + recall1*weight_tc1 + recall2*weight_tc2 + recall3*weight_tc3 + recall4*weight_tc4 + recall5*weight_tc5

precision0 = metrics.precision_score(y_actual0, y_predict0)
precision1 = metrics.precision_score(y_actual1, y_predict1)
precision2 = metrics.precision_score(y_actual2, y_predict2)
precision3 = metrics.precision_score(y_actual3, y_predict3)
precision4 = metrics.precision_score(y_actual4, y_predict4)
precision5 = metrics.precision_score(y_actual5, y_predict5)
precision = precision0*weight_tc0 + precision1*weight_tc1 + precision2*weight_tc2 + precision3*weight_tc3 + precision4*weight_tc4 + precision5*weight_tc5

accuracy0 = metrics.accuracy_score(y_actual0, y_predict0)
accuracy1 = metrics.accuracy_score(y_actual1, y_predict1)
accuracy2 = metrics.accuracy_score(y_actual2, y_predict2)
accuracy3 = metrics.accuracy_score(y_actual3, y_predict3)
accuracy4 = metrics.accuracy_score(y_actual4, y_predict4)
accuracy5 = metrics.accuracy_score(y_actual5, y_predict5)
accuracy = accuracy0*weight_tc0 + accuracy1*weight_tc1 + accuracy2*weight_tc2 + accuracy3*weight_tc3 + accuracy4*weight_tc4 + accuracy5*weight_tc5

fscore0 = metrics.f1_score(y_actual0, y_predict0)
fscore1 = metrics.f1_score(y_actual1, y_predict1)
fscore2 = metrics.f1_score(y_actual2, y_predict2)
fscore3 = metrics.f1_score(y_actual3, y_predict3)
fscore4 = metrics.f1_score(y_actual4, y_predict4)
fscore5 = metrics.f1_score(y_actual5, y_predict5)
fscore = fscore0*weight_tc0 + fscore1*weight_tc1 + fscore2*weight_tc2 + fscore3*weight_tc3 + fscore4*weight_tc4 + fscore5*weight_tc5

print('Recall:', round(recall,3), '\nPrecision:', round(precision,3), '\nAccuracy:', round(accuracy,3), '\nF1 score:', 
      round(fscore,3))

Recall: 0.673 
Precision: 0.609 
Accuracy: 0.77 
F1 score: 0.638


In [46]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# NOG VAN APARTE CLUSTERS MAKEN
conmat0 = confusion_matrix(y_actual0, y_predict0)
conmat1 = confusion_matrix(y_actual1, y_predict1)
conmat2 = confusion_matrix(y_actual2, y_predict2)
conmat3 = confusion_matrix(y_actual3, y_predict3)
conmat4 = confusion_matrix(y_actual4, y_predict4)
conmat5 = confusion_matrix(y_actual5, y_predict5)

TP = conmat0[1][1]*weight_tc0 + conmat1[1][1]*weight_tc1 + conmat2[1][1]*weight_tc2 + conmat3[1][1]*weight_tc3 + conmat4[1][1]*weight_tc4 + conmat4[1][1]*weight_tc4 + conmat5[1][1]*weight_tc5
TN = conmat0[0][0]*weight_tc0 + conmat1[0][0]*weight_tc1 + conmat2[0][0]*weight_tc2 + conmat3[0][0]*weight_tc3 + conmat4[0][0]*weight_tc4 + conmat4[0][0]*weight_tc4 + conmat5[0][0]*weight_tc5
FP = conmat0[0][1]*weight_tc0 + conmat1[0][1]*weight_tc1 + conmat2[0][1]*weight_tc2 + conmat3[0][1]*weight_tc3 + conmat4[0][1]*weight_tc4 + conmat4[0][1]*weight_tc4 + conmat5[0][1]*weight_tc5
FN = conmat0[1][0]*weight_tc0 + conmat1[1][0]*weight_tc1 + conmat2[1][0]*weight_tc2 + conmat3[1][0]*weight_tc3 + conmat4[1][0]*weight_tc4 + conmat4[1][0]*weight_tc4 + conmat5[1][0]*weight_tc5

print('TP', TP, '\nTN', TN, '\nFP', FP, '\nFN', FN)
TPshare = TP/(TP + TN + FP + FN)
TNshare = TN/(TP + TN + FP + FN)
FPshare = FP/(TP + TN + FP + FN)
FNshare = FN/(TP + TN + FP + FN)
print('TP', TPshare, '\nTN', TNshare, '\nFP', FPshare, '\nFN', FNshare)

#cm=confusion_matrix(y_actual, y_predict)

#ConfusionMatrixDisplay(confusion_matrix=cm).plot()

TP 7707.492226741479 
TN 21504.98136010585 
FP 5521.664702423248 
FN 3935.2021782275256
TP 0.19931791268122842 
TN 0.5561248549915394 
FP 0.1427917992825415 
FN 0.10176543304469068


In [47]:
print(conmat0, '\n')
print(conmat1, '\n')
print(conmat2, '\n')
print(conmat3, '\n')
print(conmat4, '\n')
print(conmat5, '\n')

[[15374  2861]
 [ 2435  5482]] 

[[19717  4043]
 [ 3300  6497]] 

[[7595  946]
 [1122 2369]] 

[[12887  2471]
 [ 2232  4452]] 

[[4316 1349]
 [ 362 2202]] 

[[32697  9856]
 [ 6528 11997]] 



In [48]:
"""[[15374  2861]
 [ 2435  5482]]
[[19717  4043]
 [ 3300  6497]]
[[7595  946]
 [1122 2369]]
[[12887  2471]
 [ 2232  4452]]"""

'[[15374  2861]\n [ 2435  5482]]\n[[19717  4043]\n [ 3300  6497]]\n[[7595  946]\n [1122 2369]]\n[[12887  2471]\n [ 2232  4452]]'

In [49]:
ttl_TP = conmat0[1][1] + conmat1[1][1] + conmat2[1][1] + conmat3[1][1] + conmat5[1][1] + conmat4[1][1] 
ttl_TN = conmat0[0][0] + conmat1[0][0] + conmat2[0][0] + conmat3[0][0]+ conmat4[0][0] + conmat5[0][0]
ttl_FP = conmat0[0][1] + conmat1[0][1] + conmat2[0][1] + conmat3[0][1]+ conmat4[0][1] + conmat5[0][1]
ttl_FN = conmat0[1][0] + conmat1[1][0] + conmat2[1][0] + conmat3[1][0]+ conmat4[1][0] + conmat5[1][0]

print(ttl_TP, ttl_TN, ttl_FP, ttl_FN)

32999 92586 21526 15979
